In [156]:
# Just take all width for viz if you have a wide screen like me.
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [143]:
import requests
from bs4 import BeautifulSoup
import re

URL = "https://www.google.com/search?"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Safari/537.36"}

title = "Dogs of War Online"
title.replace("'", "%27")

params =  {
    "as_epq" : "+".join(title.split())
}
params_str = "&".join(['='.join(x) for x in params.items()])
encoded = URL + params_str

r = requests.get(encoded, headers = headers)
r.encoding = 'utf-8'

soup = BeautifulSoup(r.content, 'html.parser')
result_stats = soup \
    .find('div', {'id' : 'result-stats'}) \
    .get_text() \
    .replace('\u202f', '') \
    .replace("\xa0", " ")

n_results = int(re.findall("[0-9]+", result_stats)[0])

print('n_results :', n_results)

n_results : 21100


In [150]:
import os
import pandas as pd

dataset_path = os.path.join(os.path.dirname(os.path.realpath("")), "data/")

df = pd.read_csv(
    dataset_path + "arstechnica/arstechnica_games_sales_1.csv"
)

df.head()

,#,APPID,TITLE,RELEASE,PLAYERS,OWNERS,TOTAL HOURS,DEVELOPER,PUBLISHER
0,1,304930.0,Unturned,7/7/2014,9099669,12226095,100073000.0,Nelson Sexton,Smartly Dressed Games
1,2,301520.0,Robocraft,7/8/2014,3549730,5344712,56569900.0,Freejam,Freejam
2,3,227940.0,Heroes & Generals,7/11/2014,2779196,4442318,22875900.0,Reto-Moto,Reto-Moto
3,4,208090.0,Loadout,1/31/2014,1993110,3312720,17126300.0,Edge of Reality,Edge of Reality
4,5,222900.0,Dead Island: Epidemic,11/20/2014,1713161,3900679,7806700.0,Stunlock Studios,Deep Silver


In [170]:
import json
import time
from tqdm.notebook import tqdm

def get_n_results(title):

    URL = "https://www.google.com/search?"
    headers = {
        "User-Agent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:57.0) Gecko/20100101 Firefox/57.0'
    }
    params =  {
        "as_epq" : "+".join(title.split())
    }
    params_str = "&".join(['='.join(x) for x in params.items()])
    encoded = URL + params_str

    print('Title :', title)
    print('Encoded url :', encoded)
    print()
    
    r = requests.get(encoded, headers = headers)
    r.encoding = 'utf-8'
    
    print("Status code:", r.status_code)

    soup = BeautifulSoup(r.content, 'html.parser')
    
    result_stats = soup \
        .find('div', {'id' : 'result-stats'}) \
        .get_text() \
        .replace('\u202f', '') \
        .replace("\xa0", " ")

    n_results = int(re.findall("[0-9]+", result_stats)[0])
    
    return n_results

# Be sure to wait before starting scraping
time.sleep(60)

res = []
for title in tqdm(df.TITLE):
    res.append((title, get_n_results(title)))
    time.sleep(60)

Title : Unturned
Encoded url : https://www.google.com/search?as_epq=Unturned

Status code: 429



AttributeError: 'NoneType' object has no attribute 'get_text'

In [176]:
r = requests.get("https://www.google.com/search?as_epq=Unturned")

print('Starting to test status code...')

max_req = 1e5
n_req = 0
while r.status_code != 200:
    time.sleep(300)
    r = requests.get("https://www.google.com/search?as_epq=Unturned")
    n_req += 1
    if n_req >= max_req:
        break
        
print('Max-Requests limit removed !')

Starting to test status code...


KeyboardInterrupt: 

In [175]:
r.headers, r.status_code

({'Date': 'Thu, 04 Feb 2021 23:08:33 GMT', 'Pragma': 'no-cache', 'Expires': 'Fri, 01 Jan 1990 00:00:00 GMT', 'Cache-Control': 'no-store, no-cache, must-revalidate', 'Content-Type': 'text/html', 'Server': 'HTTP server (unknown)', 'Content-Length': '2867', 'X-XSS-Protection': '0', 'Alt-Svc': 'h3-29=":443"; ma=2592000,h3-T051=":443"; ma=2592000,h3-Q050=":443"; ma=2592000,h3-Q046=":443"; ma=2592000,h3-Q043=":443"; ma=2592000,quic=":443"; ma=2592000; v="46,43"'},
 429)